In [41]:
#
import numpy as np
import scipy.io as sio
from keras.preprocessing import image
from skimage.transform import rotate, resize
from skimage.measure import label, regionprops
from mnet_utils import BW_img, disc_crop, mk_dir, return_list
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import sys
from glob import glob
import re
import pandas as pd

lib_path = '../optic-disc-segment/'
if lib_path not in sys.path:
    sys.path.append(lib_path)

from ipynb.fs.defs.Unet_Train import build_generator, get_dataset_mean_std, read_image_samples,get_norm_image_func, preprocess_mask,get_datagen
from ipynb.fs.defs.OD_Seg_With_ContourNet import unet_3, preprocess_MESSIDOR_mask,evaluate_model, \
    find_max_contour, show_data, dice_loss, mean_IOU_gpu, fit_ellipse, mean_IOU

disc_list = [400, 500, 600, 700, 800]
DiscROI_size = 800
CDRSeg_size = 400

data_type = '.png'
# data_img_path = '../data/REFUGE-Training400/Training400/Glaucoma/'
# label_img_path = '../data/Annotation-Training400/Annotation-Training400/Disc_Cup_Masks/Glaucoma/'
# data_img_path = '../../data/fundus-images/Baidu-GON/Training400/Glaucoma/'
# label_img_path = '../../data/fundus-images/Baidu-GON/Annotation-Training400/Disc_Cup_Masks/Glaucoma/'

data_img_path = '../../data/fundus-images/MESSIDOR/squared_image/'
label_img_path = '../../data/fundus-images/MESSIDOR/mask/'

data_save_path = mk_dir('result/MESSIDOR_crop/data/')
label_save_path = mk_dir('result/MESSIDOR_crop/label/')


file_test_list = return_list(data_img_path, data_type)

460


In [26]:
IMAGE_SIZE = (512,512)
batch_size = 1
images_dir = '../../data/fundus-images/MESSIDOR/squared_image/'
masks_dir = '../../data/fundus-images/MESSIDOR/mask/'

image_files = glob(images_dir + "*.png")
masks_files = glob(masks_dir + '*.png')
sort_method = lambda x: int(re.search('[0-9]+', x).group(0))
image_files.sort(key=sort_method)
masks_files.sort(key=sort_method)
images_df = pd.DataFrame(image_files, columns=['filename'])
masks_df = pd.DataFrame(masks_files, columns=['filename'])

In [38]:
image_samples = read_image_samples(images_df[:200])
mean,std = get_dataset_mean_std(image_samples)

img_gen_args = dict(featurewise_center=False, featurewise_std_normalization=False, preprocessing_function = get_norm_image_func(mean,std))
mask_gen_args = dict(preprocessing_function = preprocess_mask)

test_image_generator = build_generator(images_df, img_gen_args, IMAGE_SIZE, samples=image_samples)
test_mask_generator = build_generator(masks_df, mask_gen_args, IMAGE_SIZE)


test_generator = get_datagen(test_image_generator, test_mask_generator, shuffle=False, channel=[0])

[0.42642656 0.21543424 0.08097636]
[0.31391786 0.16513763 0.07199171]
Found 460 images.
Found 460 images.


In [36]:
model = unet_3()
model.load_weights('../optic-disc-segment/result/checkpoints/Unet_Disc_MESSIDOR_Best.h5')

In [39]:
evaluate_model(model, test_generator, len(test_image_generator), postprocessing=True, max_component_flag=False)

The validation IOU is: 0.937528


In [83]:
test_image_generator.shuffle = False
test_image_generator.reset()
for idx in range(len(test_image_generator)):
    temp_img = next(test_image_generator)
    img_path = test_image_generator.filenames[idx]
    img_name = os.path.split(img_path)[1]
    mask_path = os.path.join(masks_dir, img_name.replace('squared', 'mask'))
    
    org_img = np.asarray(image.load_img(img_path, target_size = (2048, 2048)))
    org_label = np.asarray(image.load_img(mask_path, target_size = (2048, 2048)))
    new_label = (org_label > 200).astype(np.uint8) * 255
    
    disc_map = model.predict(temp_img)
    disc_map = BW_img(disc_map.squeeze(), 0.5)
    
    regions = regionprops(label(disc_map))
    C_x = int(regions[0].centroid[0] * org_img.shape[0] / IMAGE_SIZE[0])
    C_y = int(regions[0].centroid[1] * org_img.shape[1] / IMAGE_SIZE[1])
    
    for disc_idx in range(len(disc_list)):
        DiscROI_size = disc_list[disc_idx]
        disc_region, err_coord, crop_coord = disc_crop(org_img, DiscROI_size, C_x, C_y)
        label_region, _, _ = disc_crop(new_label, DiscROI_size, C_x, C_y)
        Disc_flat = rotate(cv2.linearPolar(disc_region, (DiscROI_size/2, DiscROI_size/2), DiscROI_size/2,
                                           cv2.INTER_NEAREST+cv2.WARP_FILL_OUTLIERS), -90)
        Label_flat = rotate(cv2.linearPolar(label_region, (DiscROI_size/2, DiscROI_size/2), DiscROI_size/2,
                                           cv2.INTER_NEAREST+cv2.WARP_FILL_OUTLIERS), -90)

        disc_result = Image.fromarray((Disc_flat * 255).astype(np.uint8))
        disc_result.save(data_save_path + os.path.split(img_path)[1][:-4] + '_' + str(DiscROI_size) + '.png')
        label_result = Image.fromarray((Label_flat * 255).astype(np.uint8))
        label_result.save(label_save_path + os.path.split(img_path)[1][:-4] + '_' + str(DiscROI_size) + '.png')

In [ ]:
DiscSeg_model = DiscModel.DeepModel(size_set=DiscSeg_size)
DiscSeg_model.load_weights('./deep_model/Model_DiscSeg_ORIGA.h5')

for lineIdx in range(len(file_test_list)):

    temp_txt = file_test_list[lineIdx]
    print(' Processing Img ' + str(lineIdx + 1) + ': ' + temp_txt)

    # load image
    org_img = np.asarray(image.load_img(data_img_path + temp_txt))

    # load label
    org_label = np.asarray(image.load_img(label_img_path + temp_txt[:-4] + '.png'))
    new_label = org_label[org_label > 200].astype(np.uint8) 
#     new_label = np.zeros(np.shape(org_label) + (3,), dtype=np.uint8)
#     new_label[org_label < 200, 0] = 255
#     new_label[org_label < 100, 1] = 255

    # Disc region detection by U-Net
    temp_img = resize(org_img, (DiscSeg_size, DiscSeg_size, 3))*255
    temp_img = np.reshape(temp_img, (1,) + temp_img.shape)
    disc_map = DiscSeg_model.predict([temp_img])

    disc_map = BW_img(np.reshape(disc_map, (DiscSeg_size, DiscSeg_size)), 0.5)

    regions = regionprops(label(disc_map))
    C_x = int(regions[0].centroid[0] * org_img.shape[0] / DiscSeg_size)
    C_y = int(regions[0].centroid[1] * org_img.shape[1] / DiscSeg_size)

    for disc_idx in range(len(disc_list)):
        DiscROI_size = disc_list[disc_idx]
        disc_region, err_coord, crop_coord = disc_crop(org_img, DiscROI_size, C_x, C_y)
        label_region, _, _ = disc_crop(new_label, DiscROI_size, C_x, C_y)
        Disc_flat = rotate(cv2.linearPolar(disc_region, (DiscROI_size/2, DiscROI_size/2), DiscROI_size/2,
                                           cv2.INTER_NEAREST+cv2.WARP_FILL_OUTLIERS), -90)
        Label_flat = rotate(cv2.linearPolar(label_region, (DiscROI_size/2, DiscROI_size/2), DiscROI_size/2,
                                           cv2.INTER_NEAREST+cv2.WARP_FILL_OUTLIERS), -90)

        disc_result = Image.fromarray((Disc_flat * 255).astype(np.uint8))
        disc_result.save(data_save_path + temp_txt[:-4] + '_' + str(DiscROI_size) + '.png')
        label_result = Image.fromarray((Label_flat * 255).astype(np.uint8))
        label_result.save(label_save_path + temp_txt[:-4] + '_' + str(DiscROI_size) + '.png')

plt.imshow(Disc_flat)
plt.show()